In [1]:
#imports
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm as tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [2]:
# #directories
# BDSL_TRAIN = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/train'
# BDSL_VAL = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/val'
# BDSL_TEST = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/test'

# ISHARA_AUGMENETED_TRAIN = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/train'
# ISHARA_AUGMENETED_VAL = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/val'
# ISHARA_AUGMENETED_TEST = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/test'

# ISHARA_MAIN = 'F:/nbm_alvi/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_original/Isharalipi_original_main'

In [3]:
#directories

BDSL_TRAIN = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/train'
BDSL_VAL = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/val'
BDSL_TEST = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/BDSL/BDSL_split/test'

ISHARA_AUGMENTED_TRAIN = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/train'

ISHARA_AUGMENTED_VAL = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/val'
ISHARA_AUGMENTED_TEST = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_augmentated/Isharalipi_augmentated_split/test'

ISHARA_MAIN = 'E:/Bangla Sign Alphabet/Bangla Sign Alphabet Data/Isharalipi_original/Isharalipi_original_main'

In [4]:
TRAIN_DIRECTORY =ISHARA_AUGMENTED_TRAIN
VAL_DIRECTORY = ISHARA_AUGMENTED_VAL
TEST_DIRECTORY = BDSL_TEST

In [5]:
print(os.listdir(TRAIN_DIRECTORY))

['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']


In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Hyper-parameters 
num_epochs = 30
batch_size = 100
learning_rate = 0.01

In [8]:
img_transform = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

train_data = ImageFolder(TRAIN_DIRECTORY, transform=img_transform)
val_data = ImageFolder(VAL_DIRECTORY, transform=img_transform)
test_data = ImageFolder(TEST_DIRECTORY, transform=img_transform)

In [9]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

18968
2353
2381


In [10]:
train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_dl = DataLoader(test_data, batch_size*2, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_data, batch_size*2,shuffle=True, num_workers=4, pin_memory=True)

In [11]:
#load pretrained model
import torchvision.models as models

# model = models.resnet18(pretrained=False)
# model = models.alexnet(pretrained=False)
model = models.vgg16(pretrained=True)
# model = models.googlenet(pretrained=True)
# model = models.shufflenet_v2_x1_0(pretrained=False)
# model = models.mobilenet_v2(pretrained=False)
# model = models.resnext50_32x4d(pretrained=False)
# model = models.wide_resnet50_2(pretrained=False)
# model = models.mnasnet1_0(pretrained=True)

#in_features
resnet18 = 1000
alexnet = 256*6*6
vgg16 = 512*7*7
googlenet = 1024*1*1
shufflenet_v2_x1_0 = 1000
mobilenet_v2 = 1280
resnext50_32x4d = 1000
wide_resnet50_2 = 1000
mnasnet1_0 = 1280

In [14]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = nn.Sequential(nn.Linear(in_features=vgg16, out_features=512, bias=True),
                                 nn.ReLU(inplace=True),
                                 
                                 nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=512, out_features=256, bias=True),
                                 nn.ReLU(inplace=True),
                                 
                                 nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=256, out_features=35, bias=True))

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [16]:
n_total_steps = len(train_dl)
train_acc_list = []
val_acc_list = []
test_acc_list = []

train_losses = []
valid_losses = []
test_losses=[]

for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    test_loss = 0.0
    
    train_correct = 0
    train_samples = 0
    test_correct = 0
    test_samples = 0
    val_correct  = 0
    val_sample  = 0
    
    
    train_acc = 0.0
    val_acc = 0.0
    test_acc = 0.0

    model.train()
    for i, (images, labels) in enumerate(train_dl):
    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss =+ loss.item() * images.size(0)
    
    epoch_train_loss = train_loss / len(train_dl)
    train_losses.append(epoch_train_loss)
        
    #validation loss + accuracy
    with torch.no_grad():
        model.eval()
        for images, labels in val_dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            # update-average-validation-loss 
            valid_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            val_correct += torch.sum(predicted==labels.data)
            val_sample += predicted.size(0)
        
        
        epoch_val_loss = valid_loss / len(val_dl)
        val_acc_list.append((val_correct/val_sample)*100)
        valid_losses.append(epoch_val_loss)
        
    # test accuracy & loss
    with torch.no_grad():
        for images, labels in test_dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            test_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            
            test_correct += (predicted == labels).sum()
            test_samples += predicted.size(0)
        
        epoch_test_loss = test_loss / len(test_dl)
        test_acc_list.append((test_correct/test_samples)*100)
        test_losses.append(epoch_test_loss)
        
        
    # train accuracy 
    with torch.no_grad():
        for images, labels in train_dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            
            train_correct += (predicted == labels).sum()
            train_samples += predicted.size(0)
        
        train_acc_list.append((train_correct/train_samples)*100)
        



# np.savetxt("vgg_16_bn/vgg_16_bn_train_acc.txt", train_accuracy)
# np.savetxt("vgg_16_bn/vgg_16_bn_val_acc.txt", valid_accuracy)
# np.savetxt("vgg_16_bn/vgg_16_bn_test_acc.txt", test_accuracy)
# np.savetxt("vgg_16_bn/vgg_16_bn_train_loss.txt", train_loss_arr)
# np.savetxt("vgg_16_bn/vgg_16_bn_val_loss.txt", valid_loss_arr)
# np.savetxt("vgg_16_bn/vgg_16_bn_test_loss.txt", test_loss_arr)


# np.savetxt("vgg_19/vgg_19_train_acc.txt", train_accuracy)
# np.savetxt("vgg_19/vgg_19_val_acc.txt", valid_accuracy)
# np.savetxt("vgg_19/vgg_19_test_acc.txt", test_accuracy)
# np.savetxt("vgg_19/vgg_19_train_loss.txt", train_loss_arr)
# np.savetxt("vgg_19/vgg_19_val_loss.txt", valid_loss_arr)
# np.savetxt("vgg_19/vgg_19_test_loss.txt", test_loss_arr)

# np.savetxt("vgg_19_bn/vgg_19_bn_train_acc.txt", train_accuracy)
# np.savetxt("vgg_19_bn/vgg_19_bn_val_acc.txt", valid_accuracy)
# np.savetxt("vgg_19_bn/vgg_19_bn_test_acc.txt", test_accuracy)
# np.savetxt("vgg_19_bn/vgg_19_bn_train_loss.txt", train_loss_arr)
# np.savetxt("vgg_19_bn/vgg_19_bn_val_loss.txt", valid_loss_arr)
# np.savetxt("vgg_19_bn/vgg_19_bn_test_loss.txt", test_loss_arr)


# np.savetxt("resnet/resnet_train_acc.txt", train_accuracy)
# np.savetxt("resnet/resnet_val_acc.txt", valid_accuracy)
# np.savetxt("resnet/resnet_test_acc.txt", test_accuracy)
# np.savetxt("resnet/resnet_train_loss.txt", train_loss_arr)
# np.savetxt("resnet/resnet_val_loss.txt", valid_loss_arr)
# np.savetxt("resnet/resnet_test_loss.txt", test_loss_arr)

# np.savetxt("alexnet/alexnet_train_acc.txt", train_accuracy)
# np.savetxt("alexnet/alexnet_val_acc.txt", valid_accuracy)
# np.savetxt("alexnet/alexnet_test_acc.txt", test_accuracy)
# np.savetxt("alexnet/alexnet_train_loss.txt", train_loss_arr)
# np.savetxt("alexnet/alexnet_val_loss.txt", valid_loss_arr)
# np.savetxt("alexnet/alexnet_test_loss.txt", test_loss_arr)

# np.savetxt("googlenet/googlenet_train_acc.txt", train_accuracy)
# np.savetxt("googlenet/googlenet_acc.txt", valid_accuracy)
# np.savetxt("googlenet/googlenet_test_acc.txt", test_accuracy)
# np.savetxt("googlenet/googlenet_train_loss.txt", train_loss_arr)
# np.savetxt("googlenet/googlenet_val_loss.txt", valid_loss_arr)
# np.savetxt("googlenet/googlenet_test_loss.txt", test_loss_arr)


    
            
            
   




In [17]:
train_accuracy = np.array(train_acc_list)
valid_accuracy = np.array(val_acc_list)
test_accuracy = np.array(test_acc_list)
train_loss_arr = np.array(train_losses)
valid_loss_arr = np.array(valid_losses)
test_loss_arr = np.array(test_losses)

# save the list to txt file

np.savetxt("vgg_16_train_acc.txt", train_accuracy)
np.savetxt("vgg_16_val_acc.txt", valid_accuracy)
np.savetxt("vgg_16_test_acc.txt", test_accuracy)
np.savetxt("vgg_16_train_loss.txt", train_loss_arr)
np.savetxt("vgg_16_val_loss.txt", valid_loss_arr)
np.savetxt("vgg_16_test_loss.txt", test_loss_arr)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
# load txt file

# loss 
vgg_16_train_loss = np.loadtxt("vgg_16/vgg_16_train_loss.txt")
vgg_16_bn_train_loss = np.loadtxt("vgg_16_bn/vgg_16_bn_train_loss.txt")
vgg_19_train_loss = np.loadtxt("vgg_19/vgg_19_train_loss.txt")
vgg_19_bn_train_loss = np.loadtxt("vgg_19_bn/vgg_19_bn_train_loss.txt")
alexnet_train_loss = np.loadtxt("alexnet/alexnet_train_loss.txt")
resnet_train_loss = np.loadtxt("resnet/resnet_train_loss.txt")
#googlenet_train_loss = np.loadtxt("googlenet/googlenet_train_loss.txt")


# test accuracy
vgg_16_test_acc = np.loadtxt("vgg_16/vgg_16_test_acc.txt")
vgg_16_bn_test_acc= np.loadtxt("vgg_16_bn/vgg_16_bn_test_acc.txt")
vgg_19_test_acc = np.loadtxt("vgg_19/vgg_19_test_acc.txt")
vgg_19_bn_test_acc = np.loadtxt("vgg_19_bn/vgg_19_bn_test_acc.txt")
alexnet_test_acc = np.loadtxt("alexnet/alexnet_test_acc.txt")
resnet_test_acc = np.loadtxt("resnet/resnet_test_acc.txt")
#googlenet_test_acc = np.loadtxt("googlenet/googlenet_test_acc.txt")



In [ ]:
# plot

plt.plot(vgg_16_train_loss, label='vgg_16')
plt.plot(vgg_16_bn_train_loss, label='vgg_16_bn')
plt.plot(vgg_19_train_loss, label='vgg_19')
plt.plot(vgg_19_bn_train_loss, label='vgg_19_bn')
plt.plot(alexnet_train_loss, label='alexnet')
plt.plot(resnet_train_loss, label='resnet')
#plt.plot(googlenet_train_loss, label='googlenet')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [ ]:
plt.plot(vgg_16_test_acc, label='vgg_16')
plt.plot(vgg_16_bn_test_acc, label='vgg_16_bn')
plt.plot(vgg_19_test_acc, label='vgg_19')
plt.plot(vgg_19_bn_test_acc, label='vgg_19_bn')
plt.plot(alexnet_test_acc, label='alexnet')
plt.plot(resnet_test_acc, label='resnet')
#plt.plot(googlenet_test_acc, label='googlenet')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(frameon=False)